# 📖 Chapter 02 — Data Enrichment
## 🎯 Objectives
In this chapter, we will enrich our attraction data with detailed descriptions from Wikipedia.

**What we'll accomplish:**

- Set up Wikipedia API access

- Fetch Wikipedia extracts for attractions

- Clean and process description text

- Combine Geoapify data with Wikipedia content

- Create final enriched dataset for RAG

- Validate data quality and completeness


## 📦 Step 01 — Import Libraries
Import necessary libraries for working with Wikipedia API and data processing.

In [11]:
import pandas as pd
import requests
import json
import time

from src.config import PROCESSED_DATA_DIR
from src.utils.emoji_log import success, data, error, info, task, done, warn

## 🔍 Step 02 — Load Processed Data
Load the attractions with Wikipedia links from Chapter 1.

In [4]:
data_file = PROCESSED_DATA_DIR / "seattle_attractions_with_wikipedia.json"

with open(data_file, "r", encoding="utf-8") as f:
    attractions_with_wiki = json.load(f)

success(f"Loaded {len(attractions_with_wiki)} attractions with Wikipedia links")
info(f"File name: {data_file.name}")

✅ Loaded 62 attractions with Wikipedia links
💬 File name: seattle_attractions_with_wikipedia.json


## 🌐 Step 03 — Test Wikipedia API Connection
Configure Wikipedia API endpoint and test connection.

In [5]:
WIKIPEDIA_API_BASE = "https://{language}.wikipedia.org/w/api.php"

task("Testing Wikipedia API connection...")

headers = {"User-Agent": "TravelRAG/1.0 (Educational Project; dinnis1107@gmail.com)"}

test_params = {
    "action": "query",
    "format": "json",
    "titles": "Seattle",
    "prop": "extracts",
    "exintro": True,
    "explaintext": True,
}

# request test
test_response = requests.get(
    WIKIPEDIA_API_BASE.format(language="en"), params=test_params, headers=headers
)

if test_response.status_code == 200:
    success("Wikipedia API connection successful!")
else:
    error(f"Connection failed: {test_response.status_code}")

🚀 Testing Wikipedia API connection...
✅ Wikipedia API connection successful!


In [6]:
data(json.dumps(test_response.json(), indent=2))

📊 {
  "batchcomplete": "",
  "query": {
    "pages": {
      "11388236": {
        "pageid": 11388236,
        "ns": 0,
        "title": "Seattle",
        "extract": "Seattle (  see-AT-\u0259l) is the most populous city in the U.S. state of Washington and the Pacific Northwest region of North America. It is the 18th-most populous city in the United States with a population of 780,995 in 2024, while the Seattle metropolitan area at over 4.15 million residents is the 15th-most populous metropolitan area in the nation. The city is the county seat of King County, the most populous county in Washington. Seattle's growth rate of 21.1% between 2010 and 2020 made it one of the country's fastest-growing large cities.\nSeattle is situated on an isthmus between Puget Sound, an inlet of the Pacific Ocean, and Lake Washington. It is the northernmost major city in the United States, located about 100 miles (160 km) south of the Canadian border. A gateway for trade with East Asia, the Port of Seattl

## 📚 Step 04 — Fetch Wikipedia Descriptions
Fetch Wikipedia extracts for each attraction using the Wikipedia API.

In [7]:
attractions_with_wiki[0]

{'index': 0,
 'name': 'Seattle Public Library - Central Library',
 'place_id': '5186d2eaed4a955ec059a29297cfa8cd4740f00102f901ba6f35020000000092032853656174746c65205075626c6963204c696272617279202d2043656e7472616c204c696272617279',
 'wiki_data': {'wikidata': 'Q2531939',
  'wikipedia': 'en:Seattle Central Library',
  'wikimedia_commons': 'File:Seattle_(WA,_USA),_Seattle_Central_Library_--_2022_--_200930.jpg',
  'image': 'https://commons.wikimedia.org/wiki/File:Seattle_(WA,_USA),_Seattle_Central_Library_--_2022_--_200930.jpg'}}

In [8]:
task("Fetching Wikipedia descriptions for all attractions...")

success_count = 0
fail_count = 0

for i, attraction in enumerate(attractions_with_wiki, 1):
    wiki_code = attraction["wiki_data"]["wikipedia"]

    # Split the language and title
    language, title = wiki_code.split(":", maxsplit=1)

    # api url
    api_url = WIKIPEDIA_API_BASE.format(language=language)

    # params
    params = {
        "action": "query",
        "format": "json",
        "titles": title,
        "prop": "extracts",
        "exintro": True,
        "explaintext": True,
        "redirects": 1,
    }

    response = requests.get(
        api_url,
        params=params,
        headers=headers,
    )

    if response.status_code == 200:
        response_data = response.json()

        # Retrieve the description
        pages = response_data.get("query", {}).get("pages", {})

        # Iterate the pages
        description_found = False
        for page_id, page_data in pages.items():
            if "extract" in page_data and "missing" not in page_data:
                attraction["description"] = page_data["extract"]
                success_count += 1
                description_found = True
                success(f"[{i}/{len(attractions_with_wiki)}] {attraction['name']}")
                break

        if not description_found:
            attraction["description"] = None
            fail_count += 1
            error(
                f"[{i}/{len(attractions_with_wiki)}] {attraction['name']} - No extract found"
            )
    else:
        attraction["description"] = None
        fail_count += 1
        error(
            f"[{i}/{len(attractions_with_wiki)}] {attraction['name']} - API Error: {response.status_code}"
        )

    # Rate limiting
    time.sleep(0.5)

done(f"Completed! Success: {success_count}, Failed: {fail_count}")

🚀 Fetching Wikipedia descriptions for all attractions...


✅ [1/62] Seattle Public Library - Central Library
✅ [2/62] Space Needle
✅ [3/62] Pike Place Market
✅ [4/62] Seattle Great Wheel
✅ [5/62] Japanese Garden
✅ [6/62] Grass Blades
✅ [7/62] Seattle Center
✅ [8/62] Carl S. English Jr. Botanical Gardens
✅ [9/62] West Point Light
✅ [10/62] Alki Point Lighthouse
✅ [11/62] Gum Wall
✅ [12/62] Bruce and Brandon Lee Graves
✅ [13/62] Red Square
✅ [14/62] Ravenna Park Bridge
✅ [15/62] Pioneer Building
✅ [16/62] Large Lock
✅ [17/62] Small Lock
✅ [18/62] Swiftsure (LV-83)
✅ [19/62] Arthur Foss
✅ [20/62] Virginia V
✅ [21/62] Ward House
✅ [22/62] Historic Ballard Fire Station No. 18
✅ [23/62] Panama Hotel
✅ [24/62] Montlake Boulevard East
✅ [25/62] Chinatown Gate
✅ [26/62] Duwamish
✅ [27/62] A Sound Garden
✅ [28/62] Sick's Stadium
✅ [29/62] Jose Rizal Bridge
✅ [30/62] Montlake Bridge
✅ [31/62] Aurora Avenue North
✅ [32/62] Union Trust Annex
✅ [33/62] Jose Rizal Bridge
✅ [34/62] Georgetown Steam Plant
✅ [35/62] Large Lock
✅ [36/62] Mount Baker Ridge Tunnel

## 🧹 Step 05 — Clean and Process Text

Analyze data quality and clean the dataset:
- Check for duplicate entries
- Remove duplicates based on place_id
- Check for special characters
- Analyze description lengths

In [10]:
attractions_with_wiki[0]

{'index': 0,
 'name': 'Seattle Public Library - Central Library',
 'place_id': '5186d2eaed4a955ec059a29297cfa8cd4740f00102f901ba6f35020000000092032853656174746c65205075626c6963204c696272617279202d2043656e7472616c204c696272617279',
 'wiki_data': {'wikidata': 'Q2531939',
  'wikipedia': 'en:Seattle Central Library',
  'wikimedia_commons': 'File:Seattle_(WA,_USA),_Seattle_Central_Library_--_2022_--_200930.jpg',
  'image': 'https://commons.wikimedia.org/wiki/File:Seattle_(WA,_USA),_Seattle_Central_Library_--_2022_--_200930.jpg'},
 'description': 'The Seattle Central Library is the flagship library of the Seattle Public Library system. The 11-story (185 feet or 56.9 meters high) glass and steel building in the downtown core of Seattle, Washington was opened to the public on May 23, 2004. Rem Koolhaas and Joshua Prince-Ramus of OMA/LMN were the principal architects, and Magnusson Klemencic Associates was the structural engineer with Arup. Arup also provided mechanical, electrical, and plumbin

In [15]:
# Check the duplicates and remove
info("Checking for duplicates")

seen_place_ids = {}
duplicates = []

for attraction in attractions_with_wiki:
    place_id = attraction["place_id"]
    name = attraction["name"]

    if place_id in seen_place_ids:
        duplicates.append(
            {
                "name": name,
                "place_id": place_id,
                "first_index": seen_place_ids[place_id],
            }
        )
    else:
        seen_place_ids[place_id] = attraction["index"]

if duplicates:
    warn(f"Found {len(duplicates)} duplicate entries:")

    unique_attractions = []
    seen_ids = set()

    for attraction in attractions_with_wiki:
        place_id = attraction["place_id"]
        if place_id not in seen_ids:
            unique_attractions.append(attraction)
            seen_ids.add(place_id)

    attractions_with_wiki = unique_attractions
    done(
        f"Removed {len(duplicates)} duplicates. Remaining {len(attractions_with_wiki)} attractions"
    )
else:
    success("No duplicates found based on place_id")
    unique_attractions = attractions_with_wiki

💬 Checking for duplicates
✅ No duplicates found based on place_id


In [19]:
# Check for special characters

info("Checking for special characters in descriptions...")

special_char_count = 0
special_char_examples = []

for attraction in attractions_with_wiki:
    desc = attraction.get("description", "")
    if desc:
        # Check if Unicode escape sequences exists
        if any(ord(char) > 127 for char in desc): # ord -> return ordinary value
            special_char_count += 1
            special_char_examples.append(attraction["name"])

info(f"Descriptions with special characters: {special_char_count/len(attractions_with_wiki)*100:.2f}%")

if special_char_examples:
    print("Examples:")
    for name in special_char_examples:
        print(f"- {name}")

💬 Checking for special characters in descriptions...
💬 Descriptions with special characters: 27.42%
Examples:
- Red Square
- Ward House
- Panama Hotel
- Montlake Boulevard East
- Chinatown Gate
- Jose Rizal Bridge
- Montlake Bridge
- Jose Rizal Bridge
- Mount Baker Ridge Tunnel (old bore, south)
- Mount Baker Ridge Tunnel (old bore, north)
- Mount Baker Ridge Tunnel (cut and cover lid)
- Montlake Boulevard East
- Montlake Boulevard East
- Waiting for the Interurban
- 9 Spaces 9 Trees
- Broken Obelisk
- Mount Baker Ridge Tunnel (new bore)


In [28]:
# Analyze description lengths
info("Analyzing description lengths...")

desc_lengths = []
no_desc_count = 0

for attraction in attractions_with_wiki:
    desc = attraction.get("description", "")
    if desc:
        desc_lengths.append(len(desc))
    else:
        no_desc_count += 1

if desc_lengths:
    avg = sum(desc_lengths) / len(desc_lengths)
    min_length = min(desc_lengths)
    max_length = max(desc_lengths)

data("Description Length Statistics:")
print(f"  - With descriptions: {len(desc_lengths)}")
print(f"  - Without descriptions: {no_desc_count}")
print(f"  - Average length: {avg:.0f} characters")
print(f"  - Min length: {min_length} characters")
print(f"  - Max length: {max_length} characters")

💬 Analyzing description lengths...
📊 Description Length Statistics:
  - With descriptions: 62
  - Without descriptions: 0
  - Average length: 860 characters
  - Min length: 54 characters
  - Max length: 3123 characters


## 🔗 Step 06 — Merge Data
Combine Geoapify location data with Wikipedia descriptions.

## 📄 Step 07 — Create Final Documents
Format the final documents for RAG ingestion following the design from Chapter 1.

## ✅ Step 08 — Quality Validation
Validate the enriched dataset:
- Check for missing descriptions
- Verify data completeness
- Analyze description lengths
- Sample quality review

## 💾 Step 09 — Save Enriched Dataset
Save the final enriched dataset and update metadata.